In [1]:
import keras
import pandas as pd
import numpy as np

Using TensorFlow backend.


In [2]:
from keras.utils import np_utils

In [3]:
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

In [4]:
len(train_df.Event.unique())

6472

In [5]:
train_df.head()

,PID,Date,Event
0,1028890,201101,2186
1,1028890,201101,7087
2,1028890,201101,4848
3,1028890,201101,2214
4,1028890,201102,7087


In [6]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 766787 entries, 0 to 766786
Data columns (total 3 columns):
PID      766787 non-null int64
Date     766787 non-null int64
Event    766787 non-null object
dtypes: int64(2), object(1)
memory usage: 17.6+ MB


In [7]:
#train_df2 = pd.get_dummies(train_df)

In [8]:
train_df.head()

,PID,Date,Event
0,1028890,201101,2186
1,1028890,201101,7087
2,1028890,201101,4848
3,1028890,201101,2214
4,1028890,201102,7087


In [12]:
from sklearn.preprocessing import LabelEncoder
lbl = LabelEncoder()
lbl.fit(list(train_df['Event'].values)) 
train_df['Event'] = lbl.transform(list(train_df['Event'].values))
#train_cat = np_utils.to_categorical([train_df['Event']])
print('Shape train: {}\n'.format(train_df.shape))

Shape train: (766787, 3)



In [13]:
dataset=train_df.values
# split into train and test sets
train_size = int(len(dataset) * 0.67)
test_size = len(dataset) - train_size
train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]
print(len(train), len(test))

(513747, 253040)


In [14]:
Y_train = np_utils.to_categorical(train[:][1])
Y_test = np_utils.to_categorical(test[:][1])

In [15]:
Y_train

array([[ 0.,  0.,  0., ...,  0.,  0.,  1.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [16]:
Y_train.shape

(3, 1028891)

In [17]:
Y_test.shape

(3, 1016737)

In [18]:
Y_test

array([[ 0.,  0.,  0., ...,  0.,  0.,  1.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [19]:
train

array([[1028890,  201101,     657],
       [1028890,  201101,    3763],
       [1028890,  201101,    2671],
       ..., 
       [1016736,  201104,    1702],
       [1016736,  201104,    1118],
       [1016736,  201105,    1702]])

In [20]:
test

array([[1016736,  201105,    1118],
       [1016736,  201105,    1054],
       [1016736,  201106,    1568],
       ..., 
       [1022024,  201308,    5078],
       [1022024,  201308,    5095],
       [1022024,  201308,    5072]])

In [21]:
def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back), [0,2]]
        dataX.append(a)
        dataY.append(dataset[i + look_back, [2]])
    return np.array(dataX), np.array(dataY)

In [22]:
# reshape into X=t and Y=t+1
look_back = 1
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)

In [23]:
trainX

array([[[1028890,     657]],

       [[1028890,    3763]],

       [[1028890,    2671]],

       ..., 
       [[1016736,    4048]],

       [[1016736,    2190]],

       [[1016736,    1702]]])

In [24]:
trainX.shape[2]

2

In [25]:
testX.shape

(253038, 1, 2)

In [26]:
trainX.shape

(513745, 1, 2)

In [27]:
# reshape input to be [samples, time steps, features]
trainX = np.reshape(trainX, (trainX.shape[0], trainX.shape[2],1))
testX = np.reshape(testX, (testX.shape[0],  testX.shape[2],1))

In [28]:
trainX

array([[[1028890],
        [    657]],

       [[1028890],
        [   3763]],

       [[1028890],
        [   2671]],

       ..., 
       [[1016736],
        [   4048]],

       [[1016736],
        [   2190]],

       [[1016736],
        [   1702]]])

In [29]:
trainY

array([[3763],
       [2671],
       [ 678],
       ..., 
       [2190],
       [1702],
       [1118]])

In [30]:
testX

array([[[1016736],
        [   1118]],

       [[1016736],
        [   1054]],

       [[1016736],
        [   1568]],

       ..., 
       [[1022024],
        [   5089]],

       [[1022024],
        [   5058]],

       [[1022024],
        [   5078]]])

In [31]:
testY

array([[1054],
       [1568],
       [1054],
       ..., 
       [5058],
       [5078],
       [5095]])

In [34]:
#y_binary = np_utils.to_categorical(trainY)

MemoryError: 

In [32]:
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM,Embedding
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

In [38]:
# create and fit the LSTM network
model = Sequential()
model.add(Embedding(2,100))
model.add(LSTM(100, input_shape=(2, look_back),return_sequences=True))
#model.add(Dense(100,activation='linear'))
model.add(LSTM(10))
#model.add(Dense(50,activation='linear'))
#model.add(Dense(10,activation='linear'))
model.add(Dense(1))
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])
model.fit(trainX, trainY, epochs=20, batch_size=500, verbose=2)

ValueError: Error when checking input: expected embedding_3_input to have 2 dimensions, but got array with shape (513745, 2, 1)

In [ ]:
score, acc = model.evaluate(testX, testY,batch_size=100)
print('Test score:', score)
print('Test accuracy:', acc)

In [ ]:
TEST_DATA=train_df.groupby('PID').last().reset_index()

In [ ]:
TEST_DATA.head()

In [ ]:
feeding_data = TEST_DATA.iloc[:,[0,2]].values

In [ ]:
feeding_data

In [ ]:
feeding_data.shape

In [ ]:
feedX = np.reshape(feeding_data, (feeding_data.shape[0], 2,1))

In [ ]:
y_pred = model.predict(feedX)

In [ ]:
y_pred

In [ ]:
for i in range(0,len(y_pred)):
    y_pred_new[i] = lbl.inverse_transform(list(y_pred[i]))